In [2]:
import scipy
import numpy as np
from numpy import log
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from pandas import Series
import statsmodels
import sklearn
from sklearn import metrics
import statsmodels.tsa.ar_model as sta
import statsmodels.tsa.stattools as ts
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import acf, pacf
import statsmodels.api as sm
from statsmodels.tsa.api import VAR, DynamicVAR
import warnings
warnings.filterwarnings("ignore")

In [ ]:
data = pd.read_csv('TimeSeriesExamplesGasPrices.csv', index_col=0, parse_dates=True)
print(data.head())

In [ ]:
data.plot(y='Gasoline')
plt.ylabel('Price per Gallon')
plt.show()

In [ ]:
# Transform all colunms of csv into time series objects

In [ ]:
# Testing stationarity by mean
#ma = pd.rolling_mean(data, 12)
ma = data.rolling(window=12, center=False).mean()
# first 11 values should be NaN
ma[10:15]

In [ ]:
# Testing stationarity by moving standard deviation
msd = data.rolling(window=12).std()
msd[10:15]

In [ ]:
# Plot the mean and standard deviation of only ONE column
mean = plt.plot(ma.Gasoline, color='green', label='Rolling Moving Avg')
std = plt.plot(msd.Gasoline, color='red', label='Rolling Standard Dev')
plt.legend(loc='best')
plt.title('Rolling Moving Avg and St Dev')
plt.show(block=False)

In [ ]:
# Other ways to test stationarity
# Augmented Dickey-Fuller Test
# If reject H0, then it IS stationary
dataseries = Series.from_csv('TimeSeriesExamplesGasPrices.csv', header=1)
X = dataseries.values
result = adfuller(X)
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
    print('t%s: %.3f' % (key, value))
# We cannot reject the null hypothesis    

In [ ]:
# difference the data
diffX=dataseries.diff()
plt.plot(diffX)
plt.show()
diffX=diffX[1:]
diffX.head()
# plot appears more stationary

In [ ]:
# ADF test for differnced data
result = adfuller(diffX)
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))
# We reject the null hypothesis, despite the fact that variance looks uneven

In [ ]:
logdiffX=(log(dataseries)).diff()[1:]
plt.plot(logdiffX)
plt.show()

In [ ]:
# ADF test for differenced log data, 1st element removed
result = adfuller(logdiffX)
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))
# We still reject the null hypothesis, The variance looks smoother after taking log

In [ ]:
# Try to find ARIMA parameters using ACF, PACF plots
lag_acf = acf(logdiffX, nlags=20)
lag_pacf = pacf(logdiffX, nlags=20, method='ols')

In [ ]:
# Plot acf
#plt.subplot(121)
plt.plot(lag_acf)
plt.axhline(y=0,linestyle='--',color='gray')
plt.axhline(y=-1.96/np.sqrt(len(logdiffX)),linestyle='--',color='gray')
plt.axhline(y=1.96/np.sqrt(len(logdiffX)),linestyle='--',color='gray')
plt.title('Autocorrelation Function')
plt.show()

In [ ]:
# Plot pacf
#plt.subplot(121)
plt.plot(lag_pacf)
plt.axhline(y=0,linestyle='--',color='gray')
plt.axhline(y=-1.96/np.sqrt(len(logdiffX)),linestyle='--',color='gray')
plt.axhline(y=1.96/np.sqrt(len(logdiffX)),linestyle='--',color='gray')
plt.title('Partial Autocorrelation Function')
plt.show()

In [ ]:
# using AR from statsmodels to get recommended order of AR process
# Select best lag order for logdiff series    
max_lag = 10
mdl = sta.AR(logdiffX).fit(maxlag=max_lag, ic='aic', trend='nc')
est_order = sta.AR(logdiffX).select_order(maxlag=max_lag, ic='aic', trend='nc')

print('best estimated lag order = {}'.format(est_order))
print(mdl.params)
# This agrees with what ACF shows us

## Using Grid Search to estimate best ARIMA parameters

#### https://machinelearningmastery.com/grid-search-arima-hyperparameters-with-python/

In [84]:
# function to evalutate an ARIMA model for a given order(p,d,q)

def evaluate_arima_model(X, arima_order):
    # prepare training dataset
    train_size = int(len(X) * 0.66)
    train, test = X[0:train_size], X[train_size:]
    history = [x for x in train]
    # make predictions
    predictions = list()
    for t in range(len(test)):
        model = ARIMA(history, order=arima_order)
        model_fit = model.fit(disp=0)
        yhat = model_fit.forecast()[0]
        predictions.append(yhat)
        history.append(test[t])
    # calculate out of sample error
    error = sklearn.metrics.mean_squared_error(test, predictions)
    return error

In [85]:
# evaluate combinations of p, d and q values for an ARIMA model
def evaluate_models(dataset, p_values, d_values, q_values):
    dataset = dataset.astype('float32')
    best_score, best_cfg = float(10^10), None
    for p in p_values:
        for d in d_values:
            for q in q_values:
                order = (p,d,q)
                try:
                    mse = evaluate_arima_model(dataset, order)
                    if mse < best_score:
                        best_score, best_cfg = mse, order
                    print('ARIMA%s MSE=%.3f' % (order,mse))
                except:
                    continue
    print('Best ARIMA%s MSE=%.3f' % (best_cfg, best_score))

In [87]:
# set range of parameters to evaluate
p_values = [0,1,2,3,4,5]
d_values = range(0, 3)
q_values = range(0, 3)
#warnings.filterwarnings("ignore")
dataseries = Series.from_csv('TimeSeriesExamplesGasPrices.csv', header=1)
evaluate_models(dataseries.values, p_values, d_values, q_values)

ARIMA(0, 0, 0) MSE=2.006
ARIMA(0, 0, 1) MSE=0.523
ARIMA(0, 1, 0) MSE=0.004
ARIMA(0, 1, 1) MSE=0.003
ARIMA(0, 1, 2) MSE=0.003
ARIMA(0, 2, 0) MSE=0.003
ARIMA(0, 2, 1) MSE=0.003
ARIMA(0, 2, 2) MSE=0.003
ARIMA(1, 0, 0) MSE=0.004
ARIMA(1, 1, 0) MSE=0.002
ARIMA(1, 1, 1) MSE=0.002
ARIMA(1, 1, 2) MSE=0.002
ARIMA(1, 2, 0) MSE=0.003
ARIMA(2, 1, 0) MSE=0.002
ARIMA(2, 1, 1) MSE=0.002
ARIMA(2, 1, 2) MSE=0.002
ARIMA(2, 2, 0) MSE=0.003
ARIMA(3, 1, 0) MSE=0.002
ARIMA(3, 1, 1) MSE=0.002
ARIMA(3, 1, 2) MSE=0.002
ARIMA(3, 2, 0) MSE=0.003
ARIMA(4, 1, 2) MSE=0.003
ARIMA(4, 2, 0) MSE=0.003
ARIMA(5, 1, 0) MSE=0.002
ARIMA(5, 2, 0) MSE=0.003
Best ARIMANone MSE=0.000


In [ ]:
# VAR Model
vardata = Series.from_csv('TimeSeriesExamplesBirthsVar.csv', header=1)
vardata.head()
# first diffence the data
vardatadiff = diff(vardata)
# check for stationarity
result = adfuller(vardatadiff)
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
    print('t%s: %.3f' % (key, value))

In [ ]:
# build VAR model
varmdl=VAR(vardatadiff)

In [ ]:
# select the lag order, 2?
results = varmdl.fit(2)
results.summary()

In [ ]:
results.plot()

In [ ]:
 results.plot_acorr()

In [ ]:
varmdl.select_order(15)

In [ ]:
# fit using order criterion
results = model.fit(maxlags=15, ic='aic')

In [ ]:
# Forecasting
lag_order = results.k_ar
results.forecast(data.values[-lag_order:], 5)
results.plot_forecast(10)